In [1]:
# -----------------------------------------------------------
# This script serves as the main function to clean, process, and classify datasets.
#
# Author: Konghao Zhao
# Created: 2023-10-05
# Modified: 2023-11-15
# 
# -----------------------------------------------------------

In [1]:
import warnings, os
import pandas as pd
import urllib.request as ur

from sklearn.pipeline import Pipeline

from dataObject import NaColumnsHandler, NaHandler, duplicateHandler, rowOutlierChecker, correlatedFeatureRemover, normalizer, variableFeaturesSelector, pca, audit_data


ImportError: cannot import name 'NaColumnsHandler' from 'data' (/Users/konghaozhao/Documents/wfu/courses/Senior/Fall/dataMining/assignment5/scripts/data.py)

In [ ]:
################################### read data ###################################
label = "W2CL1R"
data = pd.read_csv("../data/ICPSR_36371/DS0001/36371-0001-Data.tsv", sep="\t", na_values=[" "])

#################################### data auditing ###################################
report = audit_data(data, duplicateRow = True, duplicateCol = True, NaN = True, column_NaN = True,  maxNA = 0.51)

#################################### data cleaning ###################################
print(f"{'*'*30} Data Cleaning {'*'*30}")
dataCleaner = Pipeline([
                        ("NaColumnsHandler", NaColumnsHandler(columnNames=report["Large_NAs_columns"])),
                        ("NaHandler", NaHandler(indices=report["NaN"], evaluation=False)),
                        ("logNormalizer", duplicateHandler(indices=report["duplicate"], evaluation=False)),
                        ("rowOutlierChecker", rowOutlierChecker(stdScale=3, evaluation=False)),
                        ])
data = dataCleaner.fit_transform(data)

target = data[[label]]
data = data.drop(columns=[label])

#################################### features processing ###################################
print(f"{'*'*30} Select Features {'*'*30}")
feature_selector = Pipeline([
                    ("correlatedFeatureRemover", correlatedFeatureRemover(upperBound=0.98)),
                    ("normalizer", normalizer(lowerBound=-1,upperBound=1)), 
                    ("variableFeaturesSelector", variableFeaturesSelector(numFeatures=100)),
                    ("pca", pca(criteria=0.98))
                    ])
data = feature_selector.fit_transform(data)

****************************** Data Auditing ******************************
Find 530 columns with more than 51.0% N/A
Find 3661 rows with NaN
Find 12 duplicated columns
****************************** Data Cleaning ******************************
---------- Handle Large NaN columns ----------
Drop 530 columns that have execessive N/As
Dataset shape: (3661, 257)
---------- Handle NaN values ----------
Drop 1186 rows with N/As
Dataset shape: (2475, 257)
---------- Handle duplicates ----------


KeyError: "['RECWAV2', 'PANEL120', 'INCF', 'W3CD1', 'W3VI1FLUPA', 'W3CD10W3', 'W3SEX', 'W3VO2004', 'W3VBUKE', 'W3CHVOTE'] not found in axis"